# Subsea Inpanting using Deepfill v1 ([paper](https://arxiv.org/abs/1801.07892), [code](https://github.com/JiahuiYu/generative_inpainting/tree/v1.0.0))

# 1. Setup

## 1.1. Mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.2. Link the dataset to your google drive
- You can link from [my google drive](https://drive.google.com/file/d/1OaTLKxkgKlAXMD4PFeHu4YxVVR_nqrkL/view?usp=sharing)
  - Or you can manually download the zip from google drive (or from [kaggle datasets](https://www.kaggle.com/brunomsantiago/subsea-inpainting-dataset)) and then upload to your google drive.
  - In both cases the zip should be be at your google drive root.
- Unizp the dataset using the commented code below
- You have to unzip this only once, therefore the cell bellow is commented by default

In [ ]:
# %%bash
# cd drive/MyDrive
# mkdir subsea_inpainting
# unzip subsea_inpainting_dataset.zip -d subsea_inpainting/dataset

## 1.3. Install [viajen](https://github.com/brunomsantiago/viajen) for animation visualization

In [ ]:
!pip install viajen

## 1.4. Download deepfill code and models from [subsea inpainting repository](https://github.com/brunomsantiago/subsea_inpainting)

In [ ]:
# %%bash
# cd drive/MyDrive/subsea_inpainting
# git clone https://github.com/brunomsantiago/subsea_inpainting --branch models

## 1.5. Normal imports

In [ ]:
# Standard Library imports
from pathlib import Path
import time

# Third party imports
import cv2
from IPython.core.display import HTML
import numpy as np
from PIL import Image
from viajen import animate

## 1.6 Local import (non-installed module)

In [ ]:
base_models_folder = Path('/content/drive/MyDrive/subsea_inpainting/subsea_inpainting/)
sys.path.insert(1, base_models_folder / 'code' / 'FGVC' / 'tool')
from frame_inpaint import DeepFillv1

# 2. Frames to be inpainted (+ masks)

## 2.1. Select clip to work with

In [ ]:
data_name = '01a'
# data_name = '01b'
# data_name = '01c'
# data_name = '02a'
# data_name = '03a'
# data_name = '03b'
# data_name = '04a'
# data_name = '05a'
# data_name = '05b'
# data_name = '05c'
# data_name = '05d'
# data_name = '05e'
# data_name = '05f'

## 2.2. Mount folders

In [ ]:
method = 'deepfill'
base_input_folder = '/content/drive/MyDrive/subsea_inpainting/dataset'
base_output_folder = '/content/drive/MyDrive/subsea_inpainting/results/static'

frames_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'frames'
masks_folder = Path(base_input_folder) / 'prepared_images' / data_name / 'masks'

results_folder = Path(base_output_folder) / data_name / method

## 2.3. View clip

In [ ]:
animate.folder(frames_folder, max_height=200)

In [ ]:
animate.folder(masks_folder, max_height=200)

# 3. Inpaiting

## 3.1 Weights locations

In [ ]:
deepfill_weights = base_models_folder / 'models_weights' / 'fgvc' / 'imagenet_deepfill.pth'
deepfill_weights = deepfill_weights.as_posix()

## 3.2. Function to apply inpainting

In [ ]:
def inpaint_deepfill(frames_folder, masks_folder, out_folder, model_path):
    start_time = time.time()     
    frames_files = sorted(Path(frames_folder).glob('*.png'))
    masks_files = sorted(Path(masks_folder).glob('*.png'))
    first_frame = Image.open(frames_files[0])
    deepfill = DeepFillv1(pretrained_model=model_path,
                          image_shape=[first_frame.height, first_frame.width]) 
    out_folder.mkdir(parents=True, exist_ok=True)
    for frame_file, mask_file in zip(frames_files, masks_files):
        pil_frame = Image.open(frame_file).convert('RGB') 
        frame = np.array(pil_frame) 
        frame = frame[:, :, ::-1].copy()
        mask = np.array(Image.open(mask_file))
        inpainted_frame = deepfill.forward(frame, mask)
        inpainted_frame = np.uint8(inpainted_frame)
        inpainted_frame = inpainted_frame[:, :, ::-1].copy()
        filepath = out_folder / frame_file.name
        Image.fromarray(inpainted_frame).save(filepath)
    return time.time() - start_time

## 3.3. Execute inpainting

In [ ]:
execution_time = inpaint_deepfill(frames_folder,
                                  masks_folder,
                                  results_folder,
                                  deepfill_weights)

print(f'\n\n--- Execution time: {execution_time:.2f} seconds ---')

# 4. Show results

In [ ]:
display(HTML('<h2>Original Frames<h2>'))
display(animate.folder(frames_folder))

display(HTML(f'<br><br><h2>Inpainted Frames ({execution_time:.1f} seconds processing)<h2>'))
display(animate.folder(results_folder))